In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [2]:
train_input=pd.read_csv("C:\\Users\\singh\\Desktop\\Imarticus Projects\\SVm in Python Credit Risk Analytics\\Credit_Risk_Train_data.csv")
test_input=pd.read_csv("C:\\Users\\singh\\Desktop\\Imarticus Projects\\SVm in Python Credit Risk Analytics\\Credit_Risk_Validate_data.csv")

In [3]:
print train_input.columns
print test_input.columns

Index([u'Loan_ID', u'Gender', u'Married', u'Dependents', u'Education',
       u'Self_Employed', u'ApplicantIncome', u'CoapplicantIncome',
       u'LoanAmount', u'Loan_Amount_Term', u'Credit_History', u'Property_Area',
       u'Loan_Status'],
      dtype='object')
Index([u'Loan_ID', u'Gender', u'Married', u'Dependents', u'Education',
       u'Self_Employed', u'ApplicantIncome', u'CoapplicantIncome',
       u'LoanAmount', u'Loan_Amount_Term', u'Credit_History', u'Property_Area',
       u'outcome'],
      dtype='object')


In [4]:
#the last column has a different name in both,
#lets make the names same. and then merge them togehter
#so that we can fill the missing values simulteneously
test_input.rename(columns={"outcome": "Loan_Status"},inplace=True)

In [5]:
all=pd.concat([train_input,test_input],axis=0)
all.shape

(981, 13)

In [6]:
all.head()
all.reset_index(inplace=True,drop=True)# reset index else merging will have
#issues, 

In [7]:
all.isnull().sum()#gives the missing value of all columns

Loan_ID               0
Gender               24
Married               3
Dependents           25
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [8]:
all.shape # read the decription of each column from the word document

(981, 13)

In [9]:
# before proceeding to Model Building, lets fill the missing values

In [10]:
Counter(all['Gender'])

Counter({nan: 24, 'Female': 182, 'Male': 775})

In [11]:
#lets fill them by Male
print all[all['Gender'].isnull()].index.tolist()
#these rows are null for gender
#lets fill them with the Model of Gender  i.e Male
gender_null=all[all['Gender'].isnull()].index.tolist()

[23, 126, 171, 188, 314, 334, 460, 467, 477, 507, 576, 588, 592, 636, 665, 720, 752, 823, 845, 859, 893, 910, 917, 932]


In [12]:
all['Gender'].iloc[gender_null]="Male"

In [13]:
#check if filed
print sum(all['Gender'].isnull())#oky done
Counter(all['Gender'])

0


Counter({'Female': 182, 'Male': 799})

In [14]:
#lets fill Married now
print Counter(all['Married'])#most are married
#lets fill them Yes if they have dependents else No
pd.crosstab(all['Married'].isnull(),all['Dependents'].isnull())
# for all 3 missing values in Married , the # of dependents are also missing
# let fill them with the Yes--i.e married as most customers are marrried

Counter({'Yes': 631, 'No': 347, nan: 3})


Dependents,False,True
Married,,
False,956,22
True,0,3


In [15]:
married_null=all[all['Married'].isnull()].index.tolist()
married_null

[104, 228, 435]

In [16]:
all['Married'].iloc[married_null]=np.repeat("Yes",len(married_null))

In [17]:
all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents           25
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [18]:
Counter(all['Dependents'])

Counter({nan: 25, '0': 545, '1': 160, '2': 160, '3+': 91})

In [19]:
# elts see the Dependents wrt Marriage
pd.crosstab(all['Married'],all['Dependents'].isnull())

Dependents,False,True
Married,,
No,338,9
Yes,618,16


In [20]:
pd.crosstab(all['Dependents'],all['Married'])

Married,No,Yes
Dependents,,
0,276,269
1,36,124
2,14,146
3+,12,79


In [21]:
# for the bacheors, lets fill the missing dependents as 0
#lets find the index of all rows with Depednednts mssing and Married NO
bachelor_nulldependent=all[(all['Married']=="No") & (all['Dependents'].isnull())].index.tolist()

In [22]:
all['Dependents'].iloc[bachelor_nulldependent]='0'

In [23]:
Counter(all['Dependents'])

Counter({nan: 16, '0': 554, '1': 160, '2': 160, '3+': 91})

In [24]:
#for the remaining 16 missing depemdents,
#let see how mnay dependents Male & Female have
pd.crosstab(all['Gender'],all['Dependents'])

Dependents,0,1,2,3+
Gender,,,,
Female,127,32,13,9
Male,427,128,147,82


In [25]:
# so feamle have less dependents
#lets see the gender of the 8 missing dependents
all['Gender'].iloc[all[all['Dependents'].isnull()].index.tolist()]

102      Male
104      Male
120      Male
226      Male
228      Male
301      Male
335      Male
346      Male
435    Female
517      Male
571      Male
660      Male
725      Male
816      Male
861      Male
865      Male
Name: Gender, dtype: object

In [26]:
# all of them are Male
# lets fill them with the mode of all dependent of male
pd.crosstab(all['Gender'],all['Dependents'])

Dependents,0,1,2,3+
Gender,,,,
Female,127,32,13,9
Male,427,128,147,82


In [27]:
#let sfill the # dependent with 1
all['Dependents'].iloc[all[all['Dependents'].isnull()].index.tolist()]="0"

In [28]:
all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [29]:
Counter(all['Self_Employed'])

Counter({nan: 55, 'No': 807, 'Yes': 119})

In [30]:
self_emp_null=all[all['Self_Employed'].isnull()].index.tolist()

In [31]:
#fill missing selfemployed with NO
all['Self_Employed'].iloc[self_emp_null]="No"

In [32]:
all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [33]:
pd.crosstab(all['LoanAmount'].isnull(),all['Loan_Amount_Term'])

Loan_Amount_Term,6.0,12.0,36.0,60.0,84.0,120.0,180.0,240.0,300.0,350.0,360.0,480.0
LoanAmount,,,,,,,,,,,,
False,1,2,3,3,7,4,64,7,20,1,800,22
True,0,0,0,0,0,0,2,1,0,0,23,1


In [34]:
all.groupby(all['Loan_Amount_Term'])['LoanAmount'].mean()

Loan_Amount_Term
6.0       95.000000
12.0     185.500000
36.0     117.666667
60.0     139.666667
84.0     121.142857
120.0     36.750000
180.0    131.125000
240.0    128.857143
300.0    166.250000
350.0    133.000000
360.0    144.420000
480.0    137.181818
Name: LoanAmount, dtype: float64

In [35]:
#lets fill the missing values in LoanAmount 
#with the mean of the respective Loan_Term
#we see that 180 & 240 has the almost same Loan amount 128-131
#& 360 has high i.e 144
#so lets fill only 360 by 144 
#and all remaining by 130
all['LoanAmount'][(all['LoanAmount'].isnull()) & (all['Loan_Amount_Term']==360)]=144

In [36]:
all['LoanAmount'][(all['LoanAmount'].isnull())]=130

In [37]:
#lets fill Loan Amount Term
(all['Loan_Amount_Term']).value_counts()

360.0    823
180.0     66
480.0     23
300.0     20
240.0      8
84.0       7
120.0      4
60.0       3
36.0       3
12.0       2
350.0      1
6.0        1
Name: Loan_Amount_Term, dtype: int64

In [38]:
#lets find the Loan Tenure by the mode i,e 512
all['Loan_Amount_Term'][all['Loan_Amount_Term'].isnull()]=360

In [39]:
all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [40]:
all['Credit_History'].value_counts()

1.0    754
0.0    148
Name: Credit_History, dtype: int64

In [41]:
pd.crosstab(all['Gender'],all['Credit_History']) 
# Gender makes no difference

Credit_History,0.0,1.0
Gender,,
Female,30,135
Male,118,619


In [42]:
pd.crosstab(all['Self_Employed'],all['Credit_History'])
# Self_Employed makes no difference

Credit_History,0.0,1.0
Self_Employed,,
No,134,658
Yes,14,96


In [43]:
pd.crosstab(all['Education'],all['Credit_History'])
# Education makes no difference

Credit_History,0.0,1.0
Education,,
Graduate,106,596
Not Graduate,42,158


In [44]:
pd.crosstab(all['Married'],all['Credit_History'])
# Education makes no difference

Credit_History,0.0,1.0
Married,,
No,56,263
Yes,92,491


In [45]:
# run a logistic regression to fill the Credit History for the 50 missing values

In [46]:
all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [47]:
#prepare a train set which has all values of Credit History present
#make the rows having Credit History missig as test set
#before spliting lets first create the dummies
all.columns
#df_dummy1=#for Gender
#df_dummy2=#for married
#df_dummy3=#for dependents
#df_dummy4=
#convert one columns at a time by using pd.get_dummies or use
all_new=pd.get_dummies(all.drop(['Loan_ID'],axis=1),drop_first=True)

In [48]:
all_new.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_Y
0,5849,0.0,144.0,360.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,4583,1508.0,128.0,360.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3000,0.0,66.0,360.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
3,2583,2358.0,120.0,360.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
4,6000,0.0,141.0,360.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [49]:
all_new.isnull().sum()

ApplicantIncome             0
CoapplicantIncome           0
LoanAmount                  0
Loan_Amount_Term            0
Credit_History             79
Gender_Male                 0
Married_Yes                 0
Dependents_1                0
Dependents_2                0
Dependents_3+               0
Education_Not Graduate      0
Self_Employed_Yes           0
Property_Area_Semiurban     0
Property_Area_Urban         0
Loan_Status_Y               0
dtype: int64

In [50]:
#split into train test
test=all_new[all_new['Credit_History'].isnull()]
all_in_test=test.index.tolist()
test.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_Y
16,3596,0.0,100.0,240.0,NaN,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
24,3717,2925.0,151.0,360.0,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
30,4166,3369.0,201.0,360.0,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
42,2400,0.0,75.0,360.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
79,3333,2166.0,130.0,360.0,NaN,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0


In [51]:
all_in_train=[x for x in all_new.index.tolist() if x not in all_in_test]
train=all_new.iloc[all_in_train]
train.shape

(902, 15)

In [52]:
X_train=train.drop(['Loan_Status_Y','Credit_History'],axis=1)
Y_train=train['Loan_Status_Y']

In [53]:
X_test=test.drop(['Loan_Status_Y','Credit_History'],axis=1)
Y_test=test['Loan_Status_Y']

In [54]:
#logistic regression
from sklearn.linear_model import LogisticRegression
log_reg=LogisticRegression()
log_reg.fit(X_train,Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [55]:
pred=log_reg.predict(X_test)

In [56]:
#replace the Nan's in the test with these values
test['Credit_History']=pred

In [57]:
#remerge the train & test
df_all=pd.concat([train,test],axis=0)

In [58]:
df_all.shape

(981, 15)

In [59]:
df_all.head()
df_all.isnull().sum()
#all good

ApplicantIncome            0
CoapplicantIncome          0
LoanAmount                 0
Loan_Amount_Term           0
Credit_History             0
Gender_Male                0
Married_Yes                0
Dependents_1               0
Dependents_2               0
Dependents_3+              0
Education_Not Graduate     0
Self_Employed_Yes          0
Property_Area_Semiurban    0
Property_Area_Urban        0
Loan_Status_Y              0
dtype: int64

In [60]:
#split back into the original train & test 
train2=df_all.head(len(train_input))
test2=df_all.tail(len(test_input))

In [61]:
X_train=train2.drop(['Loan_Status_Y'],axis=1)
Y_train=train2['Loan_Status_Y']
X_test=test2.drop(['Loan_Status_Y'],axis=1)
Y_test=test2['Loan_Status_Y']

# SVM in Python

In [62]:
from sklearn.svm import SVC
model=SVC()
model.fit(X_train,Y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [63]:
#above the default values of C & gamma is taken
pred=model.predict(X_test)

In [64]:
from sklearn.metrics import classification_report,confusion_matrix

In [65]:
print(classification_report(Y_test,pred))

             precision    recall  f1-score   support

        0.0       1.00      0.01      0.02        85
        1.0       0.77      1.00      0.87       282

avg / total       0.82      0.77      0.67       367



In [66]:
print(confusion_matrix(Y_test,pred))

[[  1  84]
 [  0 282]]


In [67]:
print Counter(test2['Loan_Status_Y'])
baselin_acccuracy=float(282)/(282+85)
print baselin_acccuracy
Accuracy=float(282+1)/(282+1+84)
Accuracy # not very good compared to basline

Counter({1.0: 282, 0.0: 85})
0.768392370572


0.771117166212534

In [68]:
#lets play with the C & gamma parameter in SVM
from sklearn.grid_search import GridSearchCV

In [76]:
param_grid={'C':[0.001,0.01,0.05,0.1,1,2],
           'gamma':[1,0.08,0.09,0.10,0.01],
           'kernel': ['linear']}

In [77]:
grid=GridSearchCV(SVC(),param_grid,verbose=2)

In [ ]:
grid.fit(X_train,Y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] kernel=linear, C=0.001, gamma=1 .................................
[CV] ........................ kernel=linear, C=0.001, gamma=1 - 1.1min
[CV] kernel=linear, C=0.001, gamma=1 .................................
[CV] ........................ kernel=linear, C=0.001, gamma=1 - 1.9min
[CV] kernel=linear, C=0.001, gamma=1 .................................
[CV] ........................ kernel=linear, C=0.001, gamma=1 -  54.6s
[CV] kernel=linear, C=0.001, gamma=0.08 ..............................
[CV] ..................... kernel=linear, C=0.001, gamma=0.08 - 1.2min
[CV] kernel=linear, C=0.001, gamma=0.08 ..............................
[CV] ..................... kernel=linear, C=0.001, gamma=0.08 - 1.6min
[CV] kernel=linear, C=0.001, gamma=0.08 ..............................
[CV] ..................... kernel=linear, C=0.001, gamma=0.08 -  29.1s
[CV] kernel=linear, C=0.001, gamma=0.09 ..............................
[CV] ...........

[Parallel(n_jobs=1)]: Done  40 tasks       | elapsed: 100.9min


[CV] ....................... kernel=linear, C=0.05, gamma=0.1 - 5.4min
[CV] kernel=linear, C=0.05, gamma=0.1 ................................
[CV] ....................... kernel=linear, C=0.05, gamma=0.1 - 4.1min
[CV] kernel=linear, C=0.05, gamma=0.01 ...............................
[CV] ...................... kernel=linear, C=0.05, gamma=0.01 - 3.5min
[CV] kernel=linear, C=0.05, gamma=0.01 ...............................
[CV] ...................... kernel=linear, C=0.05, gamma=0.01 - 5.4min
[CV] kernel=linear, C=0.05, gamma=0.01 ...............................
[CV] ...................... kernel=linear, C=0.05, gamma=0.01 - 4.1min
[CV] kernel=linear, C=0.1, gamma=1 ...................................
[CV] .......................... kernel=linear, C=0.1, gamma=1 - 2.1min
[CV] kernel=linear, C=0.1, gamma=1 ...................................
[CV] .......................... kernel=linear, C=0.1, gamma=1 -12.7min
[CV] kernel=linear, C=0.1, gamma=1 ...................................
[CV] .

In [ ]:
print grid.best_params_
print grid.best_estimator_

In [73]:
grid_pred=grid.predict(X_test)

In [74]:
Counter(grid_pred)

Counter({0.0: 1, 1.0: 366})

In [75]:
print(confusion_matrix(Y_test,grid_pred))

[[  1  84]
 [  0 282]]
